In [12]:
%load_ext sql
%config SqlMagic.autocommit=False # for engines that do not support autommit

# Question 1 from : https://data36.com/sql-interview-questions-tech-screening-data-analysts/

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import csv
import pandas as pd

# had issues with tables persisting using in memory 
db_connection_string = 'sqlite:///temp_db.db'

Base = declarative_base()
engine = create_engine( db_connection_string )
Base.metadata.create_all(engine)

engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Engine(sqlite:///temp_db.db)

In [13]:
# load data into tables as is 
df = pd.read_csv('./data/interview/SQL Interview Samples - authors.csv')
df.to_sql(con=engine, name='authors', if_exists='replace')

df1 = pd.read_csv('./data/interview/SQL Interview Samples - books.csv')
df1.to_sql(con=engine, name='books', if_exists='replace')


In [14]:
%sql sqlite:///temp_db.db


'Connected: @temp_db.db'

In [15]:
%%sql

select * from books
LIMIT 2;

 * sqlite:///temp_db.db
Done.


index,book_name,produced,sold %,sold,retail_price,profit,cost
0,This is a book,665397,0.6788392656000001,451697.6108,62.08352133,0.2768150884,34.15919968
1,moisterizing skin,942676,0.4591859369,432863.5623,61.05781933,0.036888031360000004,11.26025529


In [16]:
%%sql 

select * from authors
LIMIT 2;

 * sqlite:///temp_db.db
Done.


index,authors,book_name
0,Jim,This is a book
1,Jim,moisterizing skin


In [17]:
%%sql

-- Create an SQL query that shows the TOP 3 authors who sold the most books in total!

WITH books_authors AS 
( 
select *
FROM books
JOIN authors as A
ON a.book_name = books.book_name
)
select *
FROM books_authors
LIMIT 5

 * sqlite:///temp_db.db
Done.


index,book_name,produced,sold %,sold,retail_price,profit,cost,index:1,authors,book_name:1
0,This is a book,665397,0.6788392656000001,451697.6108,62.08352133,0.2768150884,34.15919968,0,Jim,This is a book
1,moisterizing skin,942676,0.4591859369,432863.5623,61.05781933,0.036888031360000004,11.26025529,1,Jim,moisterizing skin
2,firmer more balance,261050,0.3225248106,84195.10181000001,62.96783526,0.04540847454,0.9007737598000001,2,Nancy,firmer more balance
3,natrual antioxident,545257,0.7990523362999999,435688.8797,60.84349894,0.09358153646,9.770961584,3,Tracy,natrual antioxident
4,lactic acid,752786,0.7154193055,538557.6373,62.47726399,-0.03857144365,46.4014161,4,Jenna,lactic acid


In [18]:
%%sql

-- Create an SQL query that shows the TOP 3 most profitable authors so far!

WITH books_authors AS 
( 
select *, (produced * sold * retail_price * cost ) as total_profit
FROM books
JOIN authors as A
ON a.book_name = books.book_name
)

SELECT authors, SUM(total_profit)
FROM books_authors
GROUP BY authors
ORDER BY total_profit DESC


 * sqlite:///temp_db.db
Done.


authors,SUM(total_profit)
Jenna,1301142726678057.8
Jim,917946188412738.2
Tracy,141230732652221.4
C+C Music Factory,68581366190698.32
JohnWick,176765879759020.3
Liz Lemon,1305557098819841.0
Nancy,1246651355317.7036


In [19]:
## Question #2 
event_logs = pd.read_csv('./data/interview/SQL Interview Samples - event_log.csv')
event_logs.to_sql(con=engine, name='event_log', if_exists='replace')


In [20]:
%%sql

select * from event_log
LIMIT 5;

 * sqlite:///temp_db.db
Done.


index,user_id,event_date_time,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,IDs
0,19,1535308430,None,1.0,20.0,None,None,380617.0
1,16,1535308433,None,None,None,None,None,411846.0
2,4,1535308444,None,None,None,None,None,740186.0
3,3,1535308475,None,None,None,None,None,56662.0
4,6,1535308476,None,None,None,None,None,856186.0


In [21]:
%%sql

/*
    Write an SQL query to find out how many users inserted 
    more than 500 but less than 525 images in their presentations!
    first get the count by user ( from subquery ) then get count 
*/

select COUNT(*)
FROM
(select COUNT(*) as count_slides, user_id
from event_log
GROUP BY user_id)
WHERE count_slides >= 500 AND count_slides <= 525

 * sqlite:///temp_db.db
Done.


COUNT(*)
10


In [22]:
# question 3

# print out each department where average salary is is under 85k

# load data into tables as is 
salaries = pd.read_csv('./data/interview/SQL Interview Samples - salaries.csv')
salaries.to_sql(con=engine, name='salaries', if_exists='replace')

employees = pd.read_csv('./data/interview/SQL Interview Samples - employees.csv')
employees.to_sql(con=engine, name='employees', if_exists='replace')


In [28]:
%%sql

SELECT department_name, AVG(salaries) as avg_salary
FROM salaries
JOIN employees
ON salaries.employee_id = employees.employee_id
GROUP BY department_name
HAVING avg_salary < 85000

 * sqlite:///temp_db.db
Done.


department_name,avg_salary
Creative,83871.2
Marketing,80376.72727272728
Sales,80101.64285714286
